# Diamond Data

As an example, we access some available diamond data: prices in Singapore dollars and weights in carats (the standard measure of diamond mass, equal to 0.2 g). The diamond data can be downloaded from [the Journal of Statistics Education](http://jse.amstat.org/jse_data_archive.htm) and placed in the data sub directory as 'data/diamond.dat.txt').

Unlike [R](https://www.r-project.org/about.html), Python is a general-purpose language and does not have built-in features for data mining, machine learning etc. Instead, Python library developers took the core features of R (which is open source) and reimplemented them so that they can be used as function calls in Python. One of the main libraries they developed was [`pandas`](https://pandas.pydata.org/) which provides many of the same data-handling functionality of R, notably dataframes, lists, arrays, etc., with very similarly semantics to their R equivalents. This library needs to be imported first, and is given an alias `pd` for convenience.

In [ ]:
import pandas as pd
dataFile = "data/diamond.dat.txt"
import os
if not os.path.exists('res'):
    os.makedirs('res')

The next step is to read the data into the `diamondData` variable. Relative to this notebook, it can be found at `dataFile`. Whitespace is used to delimit columns in each record. The file does not include a header line with column names, so we add them on input.

In [ ]:
diamondData = pd.read_csv(dataFile, delim_whitespace=True, header=None, names=["carats","price"])

Having read the data, it is a good idea to view the leading rows, just to see its structure, etc. Note that, as with R, this is just a method call on the object (`diamondData` in this instance).

In [ ]:
diamondData.head()

It is also a good idea to plot the data, to see what it looks like. Python offers the [`matplotlib`](https://matplotlib.org/) library. We provide a directive that any plots should be put inline rather than in a separate window and then import `matplotlib` and assign an alias of `plt` to this library for convenience.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

We create a figure object and then add plot elements to it. The most component is the scatterplot generated by the `ax.scatter()` call. We can access the variables using the format `dataframe.column`. The AxesSubPlot object is just one plot ($1 \times  1$) in this case. Axis labels and a title can be added using method calls.

`plt.savefig` saves a figure, as a pdf in this case, to a file in the same directory as this Jupyter notebook.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
resFig = "res/diamonds.pdf"

ax.scatter(diamondData.carats, diamondData.price)
fig.suptitle("Relation between diamonds' price and weight")
ax.set_ylabel('Price [SIN $]')
ax.set_xlabel('Weight [carat]')
ax.grid(True)
plt.savefig(resFig)

## Fit a model

We are going to assume that diamond prices increase linearly with their weight. Having learned the parameters $ \beta_0, \beta_1 $ of that relationship, we can use them to predict the price for any weight (even one we have not seen in the training set).

Python offers the [`sklearn`](http://scikit-learn.org/stable/) library for machine learning operations, such as linear regression. However, sometimes it is helpful to use it through a more high-level "facade". In this regard, [`statsmodel`](https://github.com/statsmodels/statsmodels) provides a nice way to specify models and interact with them in a more R-like (and arguably, in a more "natural") way than is possible with `sklearn`. 

In [ ]:
import statsmodels.api as sm

We will use its function *OLS()* (Ordinary Least Squares) that fits a linear regression based on the Ordinary Least Squares algorithm.  

The model we want to get is : $\hat{y} = \beta_0 + \beta_1 X$.

where $\hat{y}$ is the estimated diamond Price (the dependent variable) and $x$ is the diamond Weight (the independent variable, called `carats` in the data).  

An intercept is not included by default and should be added when creating the $X$ matrix above.

In [ ]:
X = sm.add_constant(diamondData.carats)

Fitting the data (minimising the residual sum of squares) can be done easily using the OLS method of `statsmodel`. The fit parameters themselves $\beta_0$ and $\beta_1$ can be extracted using the 

In [ ]:
simpleModel = sm.OLS(diamondData.price, X).fit()
simpleModel.params  # here are the beta coefficients (intercept and slope of the linear regression line)

The intercept ($\beta_0 = $ {{simpleModel.params.const}}) and the slope ($\beta_1 = ${{simpleModel.params.carats}}). Thus our model is

$ \hat{y} = ${{simpleModel.params.const}} + {{simpleModel.params.carats}} $x$

where $x$ is the specific diamond Weight in carats and $y$ is the predicted diamond Price in Singapore dollars.

We can plot the obtained regression line together with the input data X.
We can do it by drawing a line using the beta parameters just calculated or also plotting the fitted values:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.scatter(diamondData.carats, diamondData.price)
# draw linear regression line
x = [0.1,0.4]
y = [simpleModel.params.const + simpleModel.params.carats * i for i in x] 
ax.plot(x, y)
# Plot the fitted values as "orange" dots for comparison with the "blue" data dots 
y_hat = simpleModel.fittedvalues
ax.scatter(diamondData.carats, y_hat)
# pretty-up the plot
fig.suptitle("Relationship between diamonds' price and weight, with OLS fit")
ax.set_ylabel('Price [SIN $]')
ax.set_xlabel('Weight [carat]')
ax.grid(True)
plt.savefig("res/diamondsFit.pdf")

## Analyse the model

### Coefficients interpretation

$\beta_1$ (the slope of the regression line) is the __expected change in response for a 1 unit change in the predictor.__  
In our case, we expect 3721 Singapore dollars increase in price for every carat increase in mass of diamond.  
This applies is within the restricted range considered; extrapolation of the regression line for bigger diamond stones would not be advisable as these stones are rarer and command a different price range.

$\beta_0$ (the intercept of the regression line) is the **expected price when the weight is zero.**
This does not always make sense and in our case the negative intercept is even more puzzling because it suggests that a zero-carat diamond ring has a negative economic value!

### Exercise: Centre the data by the mean and re-fit

Since we do not have any data on the price of diamonds with very low weights, the intercept has little meaning here. However, the intercept at the mean price would have a meaning: it is the price of diamonds at the mean weight in the sample. __You are asked to mean-centre the weight data and obtain the OLS fit to this mean-centred data.__ Hint: the centre of the weight data can be calculated using `diamondData.carats.mean()`.

__How do the $\beta$ parameters for the mean-centred data differ from those of the original data?__

## Predicting the price of a diamonds

Once we have a model of the relation, we can use it for predictions.  `statsmodel` provides a `predict()` method for this purpose.

For a single prediction, say a diamond weighing 0.2 carats, we can use

In [ ]:
newDiamond = [1, 0.2] # remember to add the intercept term (1)!
simpleModel.predict(newDiamond)

For multiple weights, we create an object with a row for each weight to be estimated. Note that the row needs to include the intercept term.

In [ ]:
newDiamonds = sm.add_constant([0.16, 0.27, 0.34]) # add the intecept for each of the 3 weights
simpleModel.predict(newDiamonds)

Result: for 0.16, 0.27, and 0.34 carats, we predict the prices to be 335.74, 745.05, 1005.52 (SIN) dollars

## Residuals

As we have seen previously, the residuals are the difference between the observed (y) and the predicted outcome (y_hat). Alternatively, they can be obtained directly from the model, using, say `simpleModel.resid`.

### Exercise: Plot the residuals and look for patterns, trends or biases

Using what you have learnt above, in relation to plotting, plot the residuals and check the following

1. they are centred on zero
2. there are no obvious patterns, e.g., runs of positive residuals followed by runs of negative residuals
3. the variance of the residuals is roughly constant over the range of prices
4. most of the residuals are near zero, but there might be a small number of more extreme values
4. there are no obvious outliers

In [ ]:
residuals = simpleModel.resid
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
resFig = "res/residuals.pdf"

ax.scatter(diamondData.carats, residuals)
fig.suptitle("Residuals plotted against weight")
ax.set_ylabel('Actual - Prediced Price [SIN $]')
ax.set_xlabel('Weight [carat]')
ax.grid(True)
plt.savefig(resFig)

In [ ]:
diamondData2 = diamondData.copy()
diamondData2.insert(1,'quality',residuals,False)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
diamondData2[['quality']] = scaler.fit_transform(diamondData2[["quality"]]).round(1)
diamondData2.head()

In [ ]:
X2 = sm.add_constant(diamondData2[["carats","quality"]])
simpleModel2 = sm.OLS(diamondData2.price, X2).fit()
simpleModel2.params

In [ ]:
residuals2 = simpleModel2.resid
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
resFig = "res/residuals2.pdf"

ax.scatter(diamondData2.carats, diamondData2.quality, residuals2)
fig.suptitle("Residuals plotted against weight and quality")
ax.set_zlabel('Actual - Prediced Price [SIN $]')
ax.set_ylabel('Quality [ratio]')
ax.set_xlabel('Weight [carat]')
ax.grid(True)
plt.savefig(resFig)

In [ ]:
fig = plt.figure()
resFig = "res/residuals3.pdf"

ax1 = plt.subplot(1, 2, 1)
ax1.scatter(diamondData2.carats, residuals2)
ax1.set_ylabel('Actual - Prediced Price [SIN $]')
ax1.set_xlabel('Weight [carat]')

ax2 = plt.subplot(1, 2, 2)
ax2.scatter(diamondData2.quality, residuals2)
ax2.set_ylabel('Actual - Prediced Price [SIN $]')
ax2.set_xlabel('Quality [ratio]')

fig.suptitle("Residuals plotted against weight")
plt.tight_layout()
plt.savefig(resFig)

### Residuals should be normally distributed

The mean of the residuals is expected to be zero, as we can check below 

In [ ]:
import numpy as np
residuals = simpleModel.resid
residMean = np.mean(residuals)
resid2Mean = np.mean(residuals2)
[residMean, resid2Mean]

The distribution of the residuals should be approximately Normal (sometimes called Gaussian) distribution.

In [ ]:
import seaborn as sns
resFig = "res/residHist.pdf"
sns_plot = sns.displot(x = residuals, kde=True)
sns_plot.savefig(resFig)

In [ ]:
resFig = "res/residHist2.pdf"
sns_plot = sns.displot(x = residuals2, kde=True)
sns_plot.savefig(resFig)

Q-Q plots (stands for a "quantile-quantile plot") can be used to check whether the data is distributed Normally or not.  

It is a plot where the axes are transformed so that a Normal (or Gaussian) distribution appears in a straight line. In other words, a __perfectly Normal distribution would exactly follow a line with slope = 1 and intercept = 0.__

If the plot does not appear to be - roughly - a straight line, then the underlying distribution is not normal. If it bends down at the left and up at the right, then there are more extreme values than expected.

The theoretical quantiles are placed along the x-axis. That is, the x-axis is not your data, it is simply an expectation of where your data should have been, if it were Normal.

The actual data is plotted along the y-axis. The values are the standard deviations from the mean. So, 0 is the mean of the data (residuals in this case), 1 is 1 standard deviation above, etc. This means, for instance, that 68.27% of all your data should be between -1 & 1, if you have a normal distribution.

`statsmodels` offers a handy `qqplot()` function:

In [ ]:
# Q-Q plot to verify the residuals distribution
resFig = "res/residualsqq.pdf"
fig = sm.qqplot(residuals, fit=True, line = '45')
fig.savefig(resFig)

In [ ]:
resFig = "res/residuals2qq.pdf"
fig = sm.qqplot(residuals2, fit=True, line = '45')
fig.savefig(resFig)

Note that the QQ plot for `residuals2` agrees with the histogram plot: the residual distribution is not Gaussian (normal) as it was for `residuals`. That is due to the way the additional (quality) variable was generated (in such a way that the distribution of the residuals is expected to be more uniform than normal, because of rounding effects).

Another thing to check is to look for observatons that have a significant influence on the model, especially those with a highy value of the residual. If such an onservation was an outlier, its removal would have a larger effect ojn the ultimate fitted model.

Interestingly, both the `simple` and `simple2` models seem to agree that observation #41 has a lot of leverage on the fit, and the size of its residual with respect to the `simple` suggests it might be an outlier.

In [ ]:
resFig = "res/simpleModel_influence.pdf"
fig = sm.graphics.influence_plot(simpleModel, criterion="cooks")
fig.tight_layout(pad=1.0)
fig.savefig(resFig)

In [ ]:
resFig = "res/simpleModel2_influence.pdf"
fig = sm.graphics.influence_plot(simpleModel2, criterion="cooks")
fig.tight_layout(pad=1.0)
fig.savefig(resFig)

In [ ]:
resFig = "res/simpleModel_carats_4plot.pdf"
fig = sm.graphics.plot_regress_exog(simpleModel, "carats")
fig.tight_layout(pad=1.0)
fig.savefig(resFig)

In [ ]:
resFig = "res/simpleModel2_carats_4plot.pdf"
fig = sm.graphics.plot_regress_exog(simpleModel2, "carats")
fig.tight_layout(pad=1.0)
fig.savefig(resFig)

In [ ]:
resFig = "res/simpleModel2_quality_4plot.pdf"
fig = sm.graphics.plot_regress_exog(simpleModel2, "quality")
fig.tight_layout(pad=1.0)
fig.savefig(resFig)

It is clear that the additional model feature `quality` does help the regression: `price` and its `fitted` value are closer, residuals show no remaining trends, etc.

This shows that the model has taken up most of the signal in the data and what is left is just "noise".

__Exercise__ Use a training-test split with each of `simpleModel` and `simpleModel2` and decide whether it is worth adding the `quality` feature in the model. 

## Measures of fit computed from the residuals 

The residual variation measures how well the regression line fits the data points.  

It is the variation in the dependent variable (Price) that is not explained by the regression model and is represented by the residuals. We want the residual variation to be as small as possible.  

Each residual is distributed normally with mean 0 and variance = $\sigma^2$.    

The __Root Mean Squared Error (RMSE)__ is defined as follows:  

In [ ]:
y = diamondData.price
n = len(y)
p = 2
df = n-p
MSE = sum(residuals**2) / df
RMSE = np.sqrt(MSE)
RMSE

RMSE can be used to calculate the standardized residuals too.  

Large standardized residuals indicate extreme values, some of which might be outliers.

In [ ]:
standardisedResiduals = simpleModel.resid / RMSE
upperExtreme = max(standardisedResiduals)
lowerExtreme = min(standardisedResiduals)
[lowerExtreme, upperExtreme]

### Summarizing the variation: R-squared

The total variation is the residual variation (variation after removing predictors) plus the systematic variation (variation explained by regression model).  

**R-squared** is the percentage of variability explained by the regression model:  

R-squared = explained / total variation = 1 - residual / total variation

R-squared is always between 0 and 1 (0% and 100%):
- 0% indicates that the model explains none of the variability of the response data around its mean.
- 100% indicates that the model explains all the variability of the response data around its mean.  

In general, the higher the R-squared, the better the model fits your data. 

In [ ]:
simpleModel.rsquared

R-squared can be a misleading summary and needs to be carefully taken (deleting data can inflate R-squared for example). Because of this, sometimes is preferred to use the **adjusted Rsquared**, which is Rsquared adjusted for the number of observations.  There are several formulas that can be used, such as Wherry's formula:

In [ ]:
1 - (1-simpleModel.rsquared)*((n-1)/simpleModel.df_resid)

Of course, it is also available from the model results:

In [ ]:
simpleModel.rsquared_adj

In conclusion, based on the residual distribution and Rsquared metric, the model is pretty good and the relation is very strong.

### Plot the confidence interval

We calculate the interval for each x value; will use the isf() function to get the inverse survival function:

In [ ]:
predicted = simpleModel.fittedvalues
x_1 = simpleModel.model.exog # this is the observation matrix used to fit the data

Get the covariance matrix of the predictors (i.e., excluding the dependent variable)

In [ ]:
covMatrixForParams = simpleModel.cov_params()
covMatrixForParams

By definition, $\mbox{var}(\hat{y}(x_0)) = \mbox{diag}(\sigma^2 x_0^T (X^TX)^{-1} x_0)$; see [this formula](https://www.otexts.org/1529). Note that $\mbox{var}(\beta) = \sigma^2 (X^TX)^{-1}$ (see Equation 30 in [these notes](https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf)). Therefore we can substitute so that $ \mbox{var}(\hat{y}(x_0)) = \mbox{diag}(x_0^T \mbox{var}(\beta) x_0)$ because $\mbox{var}(\beta) = \sigma^2 (X^TX)^{-1}$.

In [ ]:
covMatrixForPredictedPts = np.matmul(x_1, np.matmul(covMatrixForParams, x_1.T))
varPredictedPts = np.diagonal(covMatrixForPredictedPts)
varPredictedLine = simpleModel.mse_resid
totVarPredicted = varPredictedLine + varPredictedPts
totSePredicted = np.sqrt(totVarPredicted)
totSePredicted

Lookup the value of the $t$ statistic associated with $\alpha = 0.025$, using the degres of freedom from the fit = {{simpleModel.df_resid}}. 

In [ ]:
from scipy.stats import t
alpha=0.05 # confidence interval for two-sided hypothesis
qt = 1 - (alpha/2)  # (0.25, 0.975) for a 2-sided 95% probability

tppf = t.isf(alpha/2.0, simpleModel.df_resid)

Now compute the bounding confidence limits as (predicted - tppf\*totSePredicted, predicted + tppf\*totSePredicted). 

In [ ]:
interval_u = predicted + tppf * totSePredicted
interval_l = predicted - tppf * totSePredicted

In [ ]:
fig = plt.figure()
resFig = "res/smConfInt.pdf"
fig, ax = plt.subplots()
x = diamondData.carats
ax.plot(x,y, 'o', label="data")
ax.plot(x, simpleModel.fittedvalues, 'g-', label="OLS")

ax.plot(x, interval_u, 'c--', label = "Intervals")
ax.plot(x, interval_l, 'c--')

# Provide labels etc.
fig.suptitle("OLS Linear Regression with confidence intervals")
ax.set_ylabel('Predicted Price [SIN $]')
ax.set_xlabel('Weight [Carat]')
ax.grid(True)
ax.legend(loc='best')
plt.savefig(resFig)

## Summary of statistic values

The *statsmodel* package offers an overview of the model values, similar to what we calculated above:

In [ ]:
simpleModel.summary()

This table summarises the results of fitting this particular model to the data. It can be seen as having 4 sections, as follows:
1. The problem as posed to statsmodels. The `y` variable is referred to as the "dependent variable" (`price`). The model we are fitting is Ordinary Least Squares`OLS`) with the `Method` being Least Squares. The number of rows in `X` is referred to as the `No. Observations`. The number of degrees of freedom in the residuals $epsilon$ is 46, because it is calculated as the tnumber of observations (48) - the number of terms in the model (2: the slope and intercept of the line). Recall the diagram in the lecture notes where there was a grey 'space' spanned by the 2 model parameters and a given $y$ value which lay outside that space.
2. On the right hand side of this group you can see some metrics about the _overall_ fit. The F-statistic is a traditional measure of model performance - large values of the statisic, and small values of its probability are considered better. The are very encouraging here: $10^{-40}$  is effectively zero, meaning  the chances of the data happening by accident and not being generated by a model like this one are effectively zero too. The R-squared and adjusted R squared represent the ratio of the overall variance in the data that is explained by the model. Generally, anything above 0.9 is considered pretty good, so this value is very good. So it is often consiered a good idea to choose the model with the highest R-squared value. However, that can be misleading, because the "better" model (with the higher R-squared value) might overfit the data. Although they are less well-known, the _Akaike Information Criterion_ (`AIC`) and _Bayesian Information Criterion_ (`BIC`) are much better metrics to use, because they take account of the fact that models with more terms reduce the degrees of freedom and are more likely to overfit the training data, at the espense of leaving enough degrees of freedom to apply to test data. Generally, _lower_  values of AIC and BIC are considered better than larger values. Therefore, given a set of models, it is worth choosing the model with the _smallest_ value of AIC or BIC.
3. Th next table applies to the fitted paramneters $\beta$ (the slope and intercept of the line). The slope is referred to as `carats` here because that was the name of the feature we used in the model. A t-statistic is computed for each model parameter.  The most interesting feature is that its probability is so small in each case (effectively zero), suggesting that the chance that the true value of the model parameter is zero is effectively negligible. We can also see this from the fact that the confidence interval for each parameter does not include zero. The fact the fitted intercept is not zero might be surpriiong, because it suggests, for this data, that when the weight of the diamond is zero, the owner owes money! This is an artefact of the fact that it probably does not make sense to extrapolate this data down to very small diamonds. They might have their own relationship between weight and price. Extrapolation is generally  considered risk, because we are predicting values in regions of the features where we did not have any training data.
4. The next block is a set of metrics relating to how well the data and model meet the underlying assumptions for linear regression. For example, we can check whether the skewness is approximately zero (i.e., the residuals are symmetric around zero) and that kurtosis (peakiness) of the observed residual distribution is close to the value of a normal distribution (3). They appear close enough in this case; the Jarque-Bera test is a test of normality based on skewness and kurtosis and looks good here. The Durbin-Watson statistic is a check on the sequence of residuals to see whether they show _serial correlation_ (which would be unwelcome), because it suggests there is something going on in the background like: the value of the next residual can be predicted from the value of the residual before it. This is not the case here, and the Durbin-Wason takes a value very near its expected value of 2, as required. So we know from these etrics that the residuals are close to being normal, and there is no evidence of serial correlation. Lastly,  the condition number is relatively small. This metric increases as the degree of independence between the features decreases. Typically condition numbers of $10^6$ and greater might be cause for concern. There is no concern here. Overall everything looks good for theis model-data combination.

Commonly, such model summaries can be compared between models. We will see this when comparing model1 (without) and model2 (with) the `quality` feature.

Many values can also be accessed directly, for example the standard errors:

In [ ]:
simpleModel.bse

You can see all the available values using `dir()`:

In [ ]:
dir(simpleModel)

You can also see a summary of the `simpleModel2` fit

In [ ]:
simpleModel2.summary()

Compared to the R-squared value of the simpler model, which was already acceptable, it has improved even more when the `quality` feature was added. This means that model2 explains more of the variability in the data than model1. But has this come at the expense of overfitting that data? The evidence from the reduced values of `AIC` and `BIC` are that it has not: model2 is better than model because its `AIC` and `BIC` values are better (smaller). Of course we also need to check other metrics, some of which have disimporved. For example the kurtosis value is not as good (we recall from the two distribution plots that this is apparent there too) but it does not seem to matter. This shows that it is sometimes possible to improve a model by adding a term, but without seeing an improvement across all metrics. Therefore, it is always advisable to supplement any such analysis with visualisation (as done above) and also to to use techniques like cross-validation to look for evidence of over- and under-fitting.

To summarise: look for:
1. Larger is better: R-squared, Df residuals
2. Smaller is better: Probabilities: F, t, Jarque-Bera; Balanced metrics: AIC, BIC; Solver difficulty: Condition number
3. Should be close to ideal number: Skewness (0), Kurotsis (3), Durbin-Watson (2)

You cannot always improve all metrics together, so some judgment is needed. __NB you should aways use visualisation (and cross-validation to check for overfitting) to back up any model building choices.__